# intro

问题原型：Text -> Label（文本分类：X->Y）

行业baseline：
用BoW表示sentences，然后用LR或者SVM做回归。（Fan et al.2018）

深度学习是机器学习的一个分支。

库：
keras.io(theano&tf都支持)
gensim（自然语言方面）

# auto-encoder

解决问题：
1.无标签
2.标签量太大（降维）

original input-> Encoder -> compressed representation -> Decoder -> Reconstructed input

作用：
数据降噪；数据降维。




## keras安装

1.安装后端tensorflow
```
sudo pip3 install tensorflow
```

In [2]:
# 测试tensorflow安装
import tensorflow as tf
hello = tf.constant("hello, tensorflow")
sess = tf.Session()
print(sess.run(hello))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


b'hello, tensorflow'


2.安装keras
```
sudo pip3 install keras
```
3.安装sklearn
```
pip3 install scikit-learn
```

In [9]:
# 使用keras实现基于字符的auto-encoder
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.cluster import KMeans

class ASCIIAutoencoder():
    """"基于字符的auto encoder"""
    
    def __init__(self, sen_len=512, encoding_dim=32, epoch=50, val_ratio=0.3):
        """
        Init
        
        :param sen_len:把sentences pad成相同的长度
        :param encoding_dim:压缩后的维度dim
        :param epoch:要跑多少epoch
        :param kmeanmodel:简单的KNN clustering模型
        """
        self.sen_len = sen_len
        self.encoding_dim = encoding_dim
        self.autoencoder = None
        self.encoder = None
        self.kmeanmodel = KMeans(n_clusters=2)
        self.epoch = epoch
        print("inited")
        
    def preprocess(self, x, length=256):
        """
        将x截取为length长度并使用ascii表示
        """
    
    def fit(self, x):
        """
        模型构建
        :param x: input text
        """
        
        # 把所有的traintext描述成同一个size，并把每一个字符都换成ascii码
        x_train = self.preprocess(x, length=self.sen_len)
        
        # 然后给input预留好位置
        input_text = Input(shape=(self.sen_len,))
        
        # encoded 每经过一层，都被刷新成小一点的“压缩后表达式”
        encoded = Dense(1024, activation='tanh')(input_text)
        encoded = Dense(512, activation='tanh')(encoded)
        encoded = Dense(128, activation='tanh')(encoded)
        encoded = Dense(self.encoding_dim, activation='tanh')(encoded)
        
        # decoded 把刚刚压缩完成的东西，反过来还原为input_text
        decoded = Dense(128, activation='tanh')(encoded)
        decoded = Dense(512, activation='tanh')(decoded)
        decoded = Dense(1024, activation='tanh')(decoded)
        decoded = Dense(self.sen_len, activation='sigmoid')(decoded)
        
        # 整个从大到小再到大的model，叫做autoencoder
        self.autoencoder = Model(input=input_text, output=decoded)
        
        # 只从大到小叫做encoder
        self.encoder = Model(input=input_text, output=encoded)
        
        # 只从小到大是decoder
        encoded_input = Input(shape=(1024,))
        # autoencoder的最后一层应该是decoder的第一层
        decoder_layer = self.autoencoder.layers[-1]
        # 从头到尾联起来就是decoder
        decoder = Model(input=endoded_input, output=decoder_layer(encoded_input))
        
        # compile
        self.autoencoder.compile(optimizer='adam', lose='mse')
        
        # run
        self.autoencoder.fit(x_train, x_train, nb_epoch=self.epoch, batch_size=1000, shuffle=True)
        
        # 自己拿自己train一下KNN，一件简单的机遇距离的分类器
        x_train = self.encoder.predict(x_train)
        self.kmeanmodel.fit(x_train)
        
        
    def predict(self, x):
        """
        做预测
        """
        # 把输入ascii化，并且长度相等
        x_test = self.preprocess(x, length=self.sen_len)
        # 然后用encoder把test集压缩
        x_test = self.encoder.predict(x_test)
        # KNN分类
        preds = self.kmeanmodel.predict(x_test)
        
        return preds
        
if __name__ == '__main__':
    ac = ASCIIAutoencoder()

inited


# CNN4Text

1.卷积神经网络：卷积神经网络（Convolutional Neural Network,CNN）是一种前馈神经网络，它的人工神经元可以响应一部分覆盖范围内的周围单元，对于大型图像处理有出色表现。 它包括卷积层(convolutional layer)和池化层(pooling layer)。

![卷积神经网络](http://ss.csdn.net/p?https://mmbiz.qpic.cn/mmbiz_gif/heS6wRSHVMn7nLwdz2xxAfTru2GG9QaYqvsoOgfKMlRQaShOkLtyF6iaZzicXQz88dtb3BiaNCjmNzVFcHoNGiaYoA/0?wx_fmt=gif)

2.把文字表示为类似图片的形式（word2vec)

每个词都表示为一个向量，一句话就成为了一个二维的矩阵模式（类似于图片像素矩阵）。就可以变为CNN可以处理的数据格式。

## CNN案例

数据获取
RedditNews

预处理：
1. 去除非字母
2. 全小写
3. 去除stop words
4. lemma(归一化，词根还原)
5. text 2 vec
6. 模型处理


# RNN

普通神经网络：NN
有时间序列的神经网络：RNN

相比于普通神经网络：多了自我循环过程。

RNN（Recurrent Neural Network）是一类用于处理序列数据的神经网络。首先我们要明确什么是序列数据，摘取百度百科词条：时间序列数据是指在不同时间点上收集到的数据，这类数据反映了某一事物、现象等随时间的变化状态或程度。这是时间序列数据的定义，当然这里也可以不是时间，比如文字序列，但总归序列数据有一个特点——后面的数据跟前面的数据有关系。

![RNN基础模型](https://images2015.cnblogs.com/blog/1042406/201703/1042406-20170306142253375-175971779.png)

RNN可以带上记忆

# LSTM

LSTM（long short-term memory）。长短期记忆网络是RNN的一种变体，RNN由于梯度消失的原因只能有短期记忆，LSTM网络通过精妙的门控制将短期记忆与长期记忆结合起来，并且一定程度上解决了梯度消失的问题。 

![lstm](https://upload-images.jianshu.io/upload_images/42741-b9a16a53d58ca2b9.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/700)

LSTM最重要的就是cell state它一路向下，贯穿整个时间线，代表了记忆的纽带。使用XOR进行遗忘，AND进行记忆，使用Gate控制记忆或遗忘。

第一步：忘记门，决定我们会从细胞状态中丢弃什么信息。
![](https://upload-images.jianshu.io/upload_images/42741-96b387f711d1d12c.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/700)

第二步：记忆门，确定什么样的新信息被存放在细胞状态中。
![](https://img-blog.csdn.net/20160731214331118)

第三步：更新门，使用XOR和AND这样的门来更新cell state，sigmoid函数选择更新内容，tanh函数创建更新候选。
![](https://img-blog.csdn.net/20160731215529417)

第四步：输出门，确定输出什么值。
![](https://img-blog.csdn.net/20160731220103283)

案例：
1. 维度，下一个单词，下一个句子，下一个...


# 案例：用RNN做文本生成

使用LSTM，丘吉尔人物传记作为学习语料。

In [ ]:
# 第一步：导入各种库，最好用好的GPU跑，普通笔记本跑完需要一个星期吧。。。
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# 把文本读入
raw_text = open('../data/Winston_Churchil.txt').read()
raw_text = raw_text.lower()

# 用one_hot编码所有字母
chars = sorted(list(set(row_text)))

# 构造训练集

# 改造为LSTM需要的数组格式【样本数，时间步伐，特征】；output变为one-hot编码

# 构造模型

# 跑模型

# 预测